# Twitter Organizations

Imagine you have a list of organization names and you would like to know what their Twitter accounts are. This notebook explores how to look them up on Wikipedia, find the organizations home page, and then look on the homepage for a link to their Twitter account.


## Get the Wikipedia Article

First we will create a small function to get the Wikipedia Article for a given organization name. To do this we will install [requests_html](https://pypi.org/project/requests-html/) which is a nice Python library for doing HTTP requests and also for parsing returned JSON and HTML.

In [1]:
! pip install requests_html

     |████████████████████████████████| 153kB 6.9MB/s 
     |████████████████████████████████| 133kB 28.9MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for parse: filename=parse-1.17.0-cp36-none-any.whl size=24118 sha256=83182640608736893848a2075c03ad1ce2dd00dcad6a20898a7a12263336e5f5
  Stored in directory: /root/.cache/pip/wheels/83/ad/41/87f17d17fdc2b5f9648e2ec2f9dbe6ad51c2f58f086baafedf
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=60a4aec48ec3924ede8489b81b3b187e638fd6728878fa611836e3f218b0cf76
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built parse fake-useragent
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.10 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is 

Now lets create a function which we can give the organization name, and which will query the Wikipedia's [Open Search API](https://www.mediawiki.org/wiki/API:Opensearch) to get a list of articles.

In [46]:
import requests_html

http = requests_html.HTMLSession()

def get_wikipedia_article(name):
  url = 'https://en.wikipedia.org/w/api.php'
  params = {
      "action": "opensearch",
      "limit": 1,
      "namespace": 0,
      "format": "json",
      "search": name
  }
  results = http.get(url, params=params).json()
  if len(results) == 4 and len(results[1]) > 0:
    return results[3][0]
  else:
    return None  

Ok, lets test it out on *Exxon-Mobil*:

In [47]:
get_wikipedia_article('Exxon-Mobil')

'https://en.wikipedia.org/wiki/ExxonMobil'

Not bad! Lets try it with something that won't match just to make sure it doesn't throw an error.

In [48]:
get_wikipedia_article('This is a fictitious organization name!')

## Get the Official Website

Maybe there's a way to get the official website data from the Wikipedia or Wikidata API. But it's pretty easy to get the HTML for the article and look for it since it appears in a standard way.

In [131]:
def get_official_website(article_url):
  doc = http.get(article_url)
  link = doc.html.find('.official-website .url a', first=True)
  if link:
    return link.attrs['href']
  
  # fall back to looking for the first "Official website" link
  for a in doc.html.find('a'):
    if re.match('official website', a.text, re.IGNORECASE):
      return a.attrs['href']

  return None

In [129]:
get_official_website('https://en.wikipedia.org/wiki/ExxonMobil')

'http://www.exxonmobil.com'

Nice!

## Twitter Account

Now that we know the homepage for the organization we need a function that can look for a link to their Twitter account.

This is a bit more complicated because we don't know where on the page the link will be or even if there are multiple links. We do know that twitter account URLs will look something like:

    https://twitter.com/{account-name}

Where *account-name* can be any sequence of letters, numbers and underscores. Twitter used to allow you to link with a hashbang URL, which is worth looking for too eventhough it is deprecated:

    https://twitter.com/#!/{account-name}

So we can look at all the links in the page that match that pattern and return the account that has the most links.

In [113]:
import re
from collections import Counter

def get_twitter(url):

  # count all the accounts
  accounts = Counter()

  doc = http.get(url)
  for a in doc.html.find('a[href]'):
    m = re.match(r'.*twitter.com/(#!/)?([a-z0-9_]+).*', a.attrs['href'], re.IGNORECASE)
    if m:
      accounts[m.group(2)] += 1

  if len(accounts) > 0:
    return '@' + accounts.most_common()[0][0]

In [116]:
get_twitter('http://www.exxonmobil.com')

'@exxonmobil'

## Putting It Together

Now for the fun part since we can create a function that orchestrates these three functions into one that takes an organization name and returns the Twitter handle for it.

Note: we do need to take care that each step in the process returns a result in order to keep going.

In [117]:
def get_info(org_name):

  article = get_wikipedia_article(org_name)

  if article:
    homepage = get_official_website(article)
  else:
    homepage = None

  if homepage:
    twitter = get_twitter(homepage)
  else:
    twitter = None

  return {
      "article": article,
      "homepage": homepage,
      "twitter": twitter
  }


Let's try it out! 

In [118]:
get_info('Exxon Mobil')

{'article': 'https://en.wikipedia.org/wiki/ExxonMobil',
 'homepage': 'http://www.exxonmobil.com',
 'twitter': '@exxonmobil'}

🎉 Let's try it with some other organizations now. Maybe it only works for Exxon-Mobil...

In [119]:
get_info('Sears Roebuck')

{'article': 'https://en.wikipedia.org/wiki/Sears_Roebuck',
 'homepage': 'http://www.sears.com',
 'twitter': '@searsdeals'}

In [120]:
get_info('US Navy')

{'article': 'https://en.wikipedia.org/wiki/US_Navy',
 'homepage': 'http://www.navy.mil/',
 'twitter': '@usnavy'}

In [132]:
get_info("McDonalds")

{'article': 'https://en.wikipedia.org/wiki/McDonald%27s',
 'homepage': 'https://www.mcdonalds.com',
 'twitter': '@McDonalds'}

In [133]:
get_info('University of Maryland')

{'article': 'https://en.wikipedia.org/wiki/University_of_Maryland,_College_Park',
 'homepage': 'https://www.umd.edu/',
 'twitter': '@UofMaryland'}

In [135]:
get_info('Chicago Police Department')

{'article': 'https://en.wikipedia.org/wiki/Chicago_Police_Department',
 'homepage': 'https://home.chicagopolice.org/',
 'twitter': '@Chicago_Police'}